In [36]:
from python.analyse_logs import *

op_df = get_op_df(log_path)
op_df


FileNotFoundError: [Errno 2] No such file or directory: './discv5-test/logs/logs.json'

In [ ]:
op_df['reply_received'].value_counts().plot(kind = 'pie', autopct='%1.0f%%', legend=True, title='Operation returned')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots()
df = op_df[~op_df['time'].isna()]
df
sns.violinplot(x='method',y='time', data=df, ax = axes, cut=0)
#axes.set_title('Day Ahead Market')

In [ ]:
from python.analyse_logs import *

msg_df = get_msg_df(log_path, op_df)
msg_df

In [ ]:
#set the missing opids and op_types to all the messages being part of operations
#print(msg_df)
#for opid in set([i for i in op_df['opid']]):
#    print(opid)
#    op_type = op_df.loc[op_df['opid'] == opid, 'method'].values[0]
#    topic = op_df.loc[op_df['opid'] == opid, 'topic'].values[0]
#    for req_id in set([i for i in msg_df[msg_df['opid'] == opid]['req_id']]):
#        msg_df.loc[msg_df["req_id"] == req_id, "opid"] = opid
#        msg_df.loc[msg_df['req_id'] == req_id, 'op_type'] = op_type
#        msg_df.loc[msg_df['req_id'] == req_id, 'topic'] = topic


# TODO: this is very slow now! 
#msg_df

In [ ]:
from matplotlib.lines import Line2D

msg_df = msg_df.dropna(subset=['opid'])

colors = ['red', 'green', 'blue', 'yellow']
for op_type, group_op_type in msg_df.groupby('op_type'):
    #if(op_type != 'discv5_registerTopic'):
        #continue
    print(op_type)
    fig, ax = plt.subplots()
    legend_elements = []
    added = set()
    for opid, group_opid in group_op_type.groupby('opid'):
        print("\t", op_type)
        i = 0
        sum = 0
        for msg_type, group_msg_type in group_opid.groupby('msg_type'):
            val = len(group_msg_type)
            ax.bar(opid, val, color=colors[i], bottom = sum)
            sum += val
            print("\t\t", msg_type, len(group_msg_type))
            if(msg_type not in added):
                added.add(msg_type)
                legend_elements.append(Line2D([0], [0], color=colors[i], lw=4, label=msg_type))
            i += 1
    ax.legend(handles=legend_elements)
    ax.set_title(op_type)
plt.show()

In [ ]:
msg_df['in_out'].value_counts().plot(kind='pie', autopct='%1.0f%%', legend=True, title='Msgs sent/received')

In [ ]:
import matplotlib.pyplot as plt

sent = msg_df[msg_df['in_out'] == 'out']['node_id'].value_counts().to_dict()
sent = {int(k):int(v) for k,v in sent.items()} #convert IDs to int

received = msg_df[msg_df['in_out'] == 'out']['peer_id'].value_counts().to_dict()
received = {int(k):int(v) for k,v in received.items()} #convert IDs to int

width =0.3
plt.bar(sent.keys(), sent.values(), width=width, label = 'sent')
plt.bar([x + width for x in received.keys()], received.values(), width=width, label = 'received')
plt.legend()
plt.title('Messages exchanged')
plt.xlabel('Node ID')
plt.ylabel('#Messages')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots()
df_in = msg_df[msg_df['in_out']=='in']['node_id'].value_counts().rename_axis('node_id').reset_index(name='count')
df_in['in_out'] = 'in'
df_out = msg_df[msg_df['in_out']=='out']['node_id'].value_counts().rename_axis('node_id').reset_index(name='count')
df_out['in_out'] = 'out'

df = pd.concat([df_in, df_out], axis=0)
sns.violinplot(x='in_out', y='count', data=df, ax = axes, cut=0, title='#Msg received/sent distribution per node')

In [ ]:
msg_df['msg_type'].value_counts().plot(kind='bar')

In [ ]:
import numpy as np
import math
import hashlib

for op_type, group_op_type in msg_df.groupby('op_type'):
    print(op_type)
    fig, ax = plt.subplots()
    group_op_type['topic'].value_counts().plot(kind='bar', title=op_type)
    ax.set_ylabel("#Messages")



In [ ]:
op_df_exploded = op_df.copy()
op_df_exploded = op_df_exploded.explode('result')
fig, axes = plt.subplots()
op_df_exploded['result'].value_counts().plot(ax = axes,kind='bar')
axes.set_xticklabels([])
axes.set_xlabel("Discovered Nodes")
axes.set_ylabel("Count")
axes.set_yticks(list(op_df_exploded['result'].value_counts()))

In [ ]:
op_df_droppedNone = op_df_exploded.dropna(subset=['result'])
fig, axes = plt.subplots()
op_df_droppedNone['opid'].value_counts().plot(ax=axes, kind='bar')
axes.set_xlabel("Topic search operation")
axes.set_ylabel("Number of results")
axes.set_yticks(list(op_df_droppedNone['opid'].value_counts()))

In [ ]:
import numpy
#consider only final regconfig message
df = msg_df.dropna(subset=['ok'], inplace=False)
fig, ax = plt.subplots()
sns.violinplot(x='topic',y='total_wtime', data=df, ax = ax, cut = True)